Please download the sample data zip file from [here](https://s3.amazonaws.com/weclouddata/data/data/de_bd_ml_sample_data.zip) to your local. This data is used for model training. Unzip the folder, and upload the files into your Azure storage container.

In the container you connected with Databricks in the **Workshop - Mount Azure Storage**, create a new folder called `ml_training`, upload the `PostTypes.txt` and `users.csv` from your local sample data into the folder.  Then, in the `ml_training` folder, create a new subfolder, call it `Posts`, and upload all the parquet files in the local `Posts` folder into this cloud `Posts` folder.

<img src='https://s3.amazonaws.com/weclouddata/images/data_engineer/ml_tr1.jpg' width='20%'>




In this notebook, we will use machine learning -- Sentiment analysis method to train a machine learning model. The model will use to our everyday `Posts` files to tell us what each post is about (the topic of the post).

What we are going to do:
- Step 1: Join the Posts and posttypes tables, because we need to use the `Type` column in the posttypes table to filter out the posts in `Posts` table we want. And filter the data we want.
- Step 2: Prepare the training data for the machine learning training. 
- Step 3: Train the machine learning model;
- Step 4: save the model to a Azure storage folder so that you can use it for future prediction. 

## STEP 0. Loading the data into dataframes

In [0]:
# Creating a spark session
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("Table Loading")
         .getOrCreate())

sc = spark.sparkContext

#### 0.1 Creating the `Posts` dataframe

In [0]:
 
 display(
   dbutils.fs.ls("/mnt/deBDProject")
 )

path name size modificationTime dbfs:/mnt/deBDProject/landing/ landing/ 0 1715526051000 dbfs:/mnt/deBDProject/ml_training/ ml_training/ 0 1715584706000

In [0]:
# File location -- recall our mount storage workshop, data was mounted into '/mnt/deBDProject'
file_location = "/mnt/deBDProject/ml_training/Posts/*"

posts = spark.read \
  .parquet(file_location)

# display(posts)

#### 0.2 Creating the `posttypes` dataframe

In [0]:
# Creating the schema for posttypes table
from pyspark.sql.types import *

PT_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("Type", StringType(), True)
])

In [0]:
# Creating the posttypes dataframe
file_location = "/mnt/deBDProject/ml_training/PostTypes.txt"

postType = (spark.read
  .option("header", "true")
  .option("sep", ",")
  .schema(PT_schema)
  .csv(file_location))

# display(postType)

#### 0.3 Creating the `users` dataframe

In [0]:
# Creating the schema for the users table
from pyspark.sql.types import *

users_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("Age", IntegerType(), True),
    StructField("CreationDate", DateType(), True),
    StructField("DisplayName", StringType(), True),
    StructField("DownVotes", IntegerType(), True),
    StructField("EmailHash", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Reputation", IntegerType(), True),
    StructField("UpVotes", IntegerType(), True),
    StructField("Views", IntegerType(), True),
    StructField("WebsiteUrl", StringType(), True),
    StructField("AccountId", IntegerType(), True)
])

In [0]:
# Creating the users dataframe
file_location = "/mnt/deBDProject/ml_training/users.csv"

users = (spark.read
  .option("header", "true")
  .option("sep", ",")
  .schema(users_schema)
  .csv(file_location))

display(users)

id Age CreationDate DisplayName DownVotes EmailHash Location Reputation UpVotes Views WebsiteUrl AccountId 2415880 null 2021-05-24 user2415880 0 NULL NULL 1 0 0 NULL 2809323 2415881 null 2021-05-24 Julian Q. Locke 0 NULL NULL 1 0 3 NULL 1603629 2415882 null 2021-05-24 user2415882 0 NULL NULL 1 0 0 NULL 2809324 2415884 null 2021-05-24 shailesh 0 NULL NULL 1 0 0 NULL 1254325 2415885 null 2021-05-24 Sav3X 0 NULL NULL 1 0 2 null 2809327 2415886 null 2021-05-24 Jordan Schultz 0 NULL NULL 1 0 0 NULL 2809328 2415887 null 2021-05-24 user2415887 0 NULL NULL 1 0 1 NULL 2809329 2415888 null 2021-05-24 luvrei 0 NULL NULL 1 0 3 null 2809330 2415889 null 2021-05-24 Lee24 0 NULL NULL 1 0 0 null 2809326 2415890 null 2021-05-24 broilogabriel 0 NULL Dublin, Ireland 15 16 6 null 2809331 2415891 null 2021-05-24 whittie83 0 NULL Melbourne, Australia 8 0 5 null 2809332 2415892 null 2021-05-24 Dk.Pure 0 NULL NULL 1 0 0 null 2809335 2415893 null 2021-05-24 user2415893 0 NULL NULL 1 0 1 NULL 2809336 2415894 null 2021-05-24 Edinson Viggio 0 NULL NULL 1 0 0 NULL 2809337 2415895 null 2021-05-24 user2415895 0 NULL NULL 11 0 1 NULL 2809338 2415896 null 2021-05-24 V Anton Spraul 0 NULL NULL 66 0 15 NULL 2809339 2415897 null 2021-05-17 Counter10000 0 NULL NULL 89 402 38 null 2781822 2415898 null 2021-05-24 CENTRI 0 NULL NULL 31 0 3 NULL 2809341 2415899 null 2021-05-24 juanriccio 0 NULL NULL 11 0 2 null 2809342 2415900 null 2021-05-24 Deepak Singh 0 NULL India 197 10 120 null 2809343 2415901 null 2021-05-24 user2415901 0 NULL NULL 1 0 0 NULL 2809344 2415902 null 2021-05-24 user2415902 0 NULL NULL 1 0 0 NULL 2809346 2415903 null 2021-05-24 user2415903 0 NULL NULL 1 0 1 NULL 2809348 2415904 null 2021-05-24 Chris 0 NULL NULL 11 0 0 NULL 2809349 2415905 null 2021-05-24 aerosjc 0 NULL NULL 1 0 1 null 2809351 2415906 null 2021-05-24 hassicho 0 NULL NULL 69 11 10 null 2809352 2415907 null 2021-05-24 user2415907 0 NULL NULL 1 0 1 NULL 2809353 2415908 null 2021-05-24 Tasuku Okuda 0 NULL Tokyo, Japan 1 0 0 NULL 2809355 2415909 null 2021-05-24 user2415909 0 NULL NULL 1 0 0 NULL 2809356 2415910 null 2021-05-24 dearborn 0 NULL NULL 1 0 0 NULL 2262718 2415911 null 2021-05-24 zmitchell 0 NULL NULL 6 0 0 null 2809357 2415912 null 2021-05-24 user2415912 0 NULL NULL 9 0 11 NULL 2809358 2415913 null 2021-05-24 Nick Price 0 NULL NULL 468 87 95 NULL 2809360 2415914 null 2021-05-24 iekwei 0 NULL NULL 96 0 7 NULL 2809361 2415915 null 2021-05-24 janchiks 0 NULL China 1 0 0 null 2809363 2415916 null 2021-05-24 Ashley Djashb Houck 0 NULL NULL 1 0 0 NULL 2809364 2415917 null 2021-05-24 user2415917 0 NULL NULL 1 0 0 NULL 2809366 2415918 null 2021-05-24 Koji Miyauchi 0 NULL NULL 1 0 5 NULL 2809367 2415919 null 2021-05-24 Mou 0 NULL NULL 103 0 32 null 2809369 2415920 null 2021-05-24 tdebat 0 NULL NULL 1 0 2 null 2809370 2415921 null 2021-05-24 Michael 45 NULL Grand Rapids, MI, United States 4609 3652 324 http://mikemilla.com 2809371 2415922 null 2021-05-24 user2415922 0 NULL NULL 1 0 1 NULL 2809372 2415923 null 2021-05-24 fif4evr 0 NULL NULL 1 0 1 null 2809376 2415924 null 2021-05-24 user2415924 0 NULL NULL 16 0 1 NULL 2809377 2415925 null 2021-05-24 user2415925 0 NULL NULL 1 0 0 NULL 2809380 2415926 null 2021-05-24 virginie 0 NULL NULL 1 0 0 NULL 2809381 2415927 null 2021-05-24 user2415927 0 NULL NULL 18 0 4 null 2809382 2415928 null 2021-05-24 user2415928 0 NULL NULL 1 0 0 NULL 2809383 2415929 null 2021-05-24 user2415929 0 NULL NULL 1 0 0 NULL 2809385 2415930 null 2021-05-24 user2415930 0 NULL NULL 1 0 0 NULL 2809387 2415931 null 2021-05-24 Jorge Santamaria 0 NULL NULL 58 0 7 null 2809389 2415932 null 2021-05-24 InternetFett 0 NULL NULL 1 0 2 null 2809390 2415933 null 2021-05-24 user2415933 0 NULL NULL 1 0 0 NULL 2809391 2415934 null 2021-05-24 yangzx 0 NULL NULL 21 0 3 null 2809392 2415935 null 2021-05-24 user2415935 0 NULL NULL 1 0 0 NULL 2809393 2415936 null 2021-05-24 qgymje 0 NULL NULL 1 0 5 null 2809394 2415938 null 2021-05-24 mesa1014 0 NULL NULL 1 0 0 null 2809397 2415939 null 2

#### 0.4. Saving the dataframes for easy retrieval

In [0]:
# Save the 3 tables to databricks local file system
posts.write.parquet("/tmp/project/posts.parquet")
postType.write.parquet("/tmp/project/PostType.parquet")
users.write.parquet("/tmp/project/user.parquet")

In [0]:
display(dbutils.fs.ls("/tmp/project/"))

path name size modificationTime dbfs:/tmp/project/PostType.parquet/ PostType.parquet/ 0 1715586320000 dbfs:/tmp/project/posts.parquet/ posts.parquet/ 0 1715586317000 dbfs:/tmp/project/user.parquet/ user.parquet/ 0 1715586320000

## STEP 1. Join tables and filter data

#### 1.1 Prepare necessary libraries and load data

In [0]:
# Import necessary libraries and functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, translate, trim, explode, regexp_replace, col, lower

In [0]:
# Creating Spark Session
spark = (SparkSession
         .builder
         .appName("ML Model")
         .getOrCreate())

sc = spark.sparkContext

In [0]:
# Read in the tables

posts = spark.read.parquet("/tmp/project/posts.parquet")
postType = spark.read.parquet("/tmp/project/PostType.parquet")
Users = spark.read.parquet("/tmp/project/user.parquet")

#### 1.2 Join the tables Posts and postTypes by it post type id

In [0]:
# at this moment, we only use Posts and posttypes to train the model. so let's join them iwith the posttype id. 

df = posts.join(postType, posts.PostTypeId == postType.id)
# display(df)

In [0]:
display(df)

id AcceptedAnswerId AnswerCount Body CommentCount CreationDate FavoriteCount LastEditDate LastEditorDisplayName LastEditorUserId OwnerUserId ParentId PostTypeId Score Tags Title ViewCount id Type 6717493 6717498 2 I have an abstract class with a protected variable abstract class Beverage { protected string description; } I can't access it from a subclass. Intellisense doesn't show it accessible. Why is that so? class Espresso:Beverage { //this.description ?? } 4 2023-07-16T00:00:00Z 1 2023-08-10T00:00:00Z NULL 105084 205859 0 1 2.0 List(c#, .net, abstract-class, protected) Why can't I access protected variable in subclass? 3791 1 Question 6800771 6801823 3 I have an input of a list of pairs: [[abs_(p,X,Y,Z),abs_(f,X,Y,Z)],[abs_(p,X,Y,Z),abs_(l,Z,P)]] I want check if a pair have the same number of arguments, in this case yes: [abs_(p,X,Y,Z),abs_(f,X,Y,Z)] In the second case the answer is no. This is just an example because more generally, I want to know which pair have the same number of arguments. The output for the input should be: [[abs_(p,X,Y,Z),abs_(f,X,Y,Z)] What do I have to do? 0 2023-07-23T00:00:00Z 0 2023-08-10T00:00:00Z NULL 373694 448381 0 1 1.0 List(list, prolog) prolog list of pairs 3364 1 Question 6450616 6450648 3 "How can I count the matched classes with jQuery's :contains? If I have the following code : <!DOCTYPE html> <html> <head> <script src=""http://code.jquery.com/jquery-latest.js""></script> </head> <body> <div class=""ll"">John Resig</div> <div class=""ll"">George Martin</div> <div>Malcom John Sinclair</div> <div class=""ll"">J. Ohn</div> <script> $(""div.ll"").hide() $(""div.ll:contains('John')"").show(); </script> </body> </html> How do I count the matched elements? " 0 2023-06-23T00:00:00Z 0 2023-08-10T00:00:00Z NULL 327466 632316 0 1 0.0 List(jquery, jquery-selectors) How to count elements matched with :contains? 2890 1 Question 6832220 6841293 1 I'm trying to bind a list/arraylist/hashmap/etc of custom objects to my form in JSP using Spring. Right now, the controller creates a Map of the two lists (Boolean list and custom object list) in referenceData(), and provides it to the form which uses those values to populate the fields. The values are initialized from a MySQL database using Hibernate, and all that works fine. The list is a known length before the form is initialized, so that part is easier. Now what I'd like to do is correctly bind those objects in the form, so that when there are changes made, I can detect that in onSubmit() (or wherever is appropriate), and update the database accordingly. I can't seem to bind them correctly in the form so that I can see changes made. I tried just using a list of the form fields as the model, but even that wasn't working correctly. Do I just need to inject the list in a particular way? Any ideas or examples here? Any help would be greatly appreciated. UPDATE: At Ralph's request here is the solution I used: In my data object class, I lazy loaded a map using MapUtils.lazyMap(), with a String key and other custom object value. The other custom object is just a class that contains List<String> and getters/setters. In the corresponding .jsp file, I just nest several loops to loop through the keys first using loop.current.key and then loop2.current.value.paramsList to loop through the values for that key. This was not really what I asked for in my original post, as I was looking for a more general solution, and the lazy loading pointed me in the right direction. 3 2023-07-26T00:00:00Z 0 2023-08-10T00:00:00Z NULL 483402 483402 0 1 0.0 List(forms, spring, jsp, binding) Spring List Binding in Form 572 1 Question 6840579 6840643 2 I am using tabcontainer control of Ajaxtoolkit.I want to load aspx page for each of the tabs asynchronously i.e when i click on tab at that time an aspx page should be loaded 0 2023-07-27T00:00:00Z 0 2023-08-10T00:00:00Z NULL 97010 367339 0 1 0.0 List(asp.net, html, ajax, tabcontrol) How to load aspx pages in tabcontainer 2372 1 Question

#### 1.3 Filter the data

In the posttypes table, there is a column called `Type` which indicates if the posts is a question or an answer. We only need the 'question' entires. For these 'Question' rows, we will run machine learning model on the join the 'Body' column of the 'Posts' table. To tell what topic this post is about.

In [0]:
# Filter the dataframe to only include questions
df = df.filter(col("Type") == "Question")
display(df)

id AcceptedAnswerId AnswerCount Body CommentCount CreationDate FavoriteCount LastEditDate LastEditorDisplayName LastEditorUserId OwnerUserId ParentId PostTypeId Score Tags Title ViewCount id Type 6717493 6717498 2 I have an abstract class with a protected variable abstract class Beverage { protected string description; } I can't access it from a subclass. Intellisense doesn't show it accessible. Why is that so? class Espresso:Beverage { //this.description ?? } 4 2023-07-16T00:00:00Z 1 2023-08-10T00:00:00Z NULL 105084 205859 0 1 2.0 List(c#, .net, abstract-class, protected) Why can't I access protected variable in subclass? 3791 1 Question 6800771 6801823 3 I have an input of a list of pairs: [[abs_(p,X,Y,Z),abs_(f,X,Y,Z)],[abs_(p,X,Y,Z),abs_(l,Z,P)]] I want check if a pair have the same number of arguments, in this case yes: [abs_(p,X,Y,Z),abs_(f,X,Y,Z)] In the second case the answer is no. This is just an example because more generally, I want to know which pair have the same number of arguments. The output for the input should be: [[abs_(p,X,Y,Z),abs_(f,X,Y,Z)] What do I have to do? 0 2023-07-23T00:00:00Z 0 2023-08-10T00:00:00Z NULL 373694 448381 0 1 1.0 List(list, prolog) prolog list of pairs 3364 1 Question 6450616 6450648 3 "How can I count the matched classes with jQuery's :contains? If I have the following code : <!DOCTYPE html> <html> <head> <script src=""http://code.jquery.com/jquery-latest.js""></script> </head> <body> <div class=""ll"">John Resig</div> <div class=""ll"">George Martin</div> <div>Malcom John Sinclair</div> <div class=""ll"">J. Ohn</div> <script> $(""div.ll"").hide() $(""div.ll:contains('John')"").show(); </script> </body> </html> How do I count the matched elements? " 0 2023-06-23T00:00:00Z 0 2023-08-10T00:00:00Z NULL 327466 632316 0 1 0.0 List(jquery, jquery-selectors) How to count elements matched with :contains? 2890 1 Question 6832220 6841293 1 I'm trying to bind a list/arraylist/hashmap/etc of custom objects to my form in JSP using Spring. Right now, the controller creates a Map of the two lists (Boolean list and custom object list) in referenceData(), and provides it to the form which uses those values to populate the fields. The values are initialized from a MySQL database using Hibernate, and all that works fine. The list is a known length before the form is initialized, so that part is easier. Now what I'd like to do is correctly bind those objects in the form, so that when there are changes made, I can detect that in onSubmit() (or wherever is appropriate), and update the database accordingly. I can't seem to bind them correctly in the form so that I can see changes made. I tried just using a list of the form fields as the model, but even that wasn't working correctly. Do I just need to inject the list in a particular way? Any ideas or examples here? Any help would be greatly appreciated. UPDATE: At Ralph's request here is the solution I used: In my data object class, I lazy loaded a map using MapUtils.lazyMap(), with a String key and other custom object value. The other custom object is just a class that contains List<String> and getters/setters. In the corresponding .jsp file, I just nest several loops to loop through the keys first using loop.current.key and then loop2.current.value.paramsList to loop through the values for that key. This was not really what I asked for in my original post, as I was looking for a more general solution, and the lazy loading pointed me in the right direction. 3 2023-07-26T00:00:00Z 0 2023-08-10T00:00:00Z NULL 483402 483402 0 1 0.0 List(forms, spring, jsp, binding) Spring List Binding in Form 572 1 Question 6840579 6840643 2 I am using tabcontainer control of Ajaxtoolkit.I want to load aspx page for each of the tabs asynchronously i.e when i click on tab at that time an aspx page should be loaded 0 2023-07-27T00:00:00Z 0 2023-08-10T00:00:00Z NULL 97010 367339 0 1 0.0 List(asp.net, html, ajax, tabcontrol) How to load aspx pages in tabcontainer 2372 1 Question

In [0]:
# Filter the dataframe to only include questions
df = df.filter(col("Type") == "Question")
# display(df)

#### 1.4 Create a checkpoint to save the dataframe to file only contain the `Body` and `Tag` we need. 

In [0]:
df = df.select(col("Body").alias("text"), col("Tags"))

In [0]:
# Producing the tags as individual tags instead of an array
# This is duplicating the posts for each possible tag
df = df.select("text", explode("Tags").alias("tags"))
# display(df)

In [0]:
# saving the file as a checkpoint (in case the cluster gets terminated)

df.write.parquet("/tmp/project.df.parquet")

In [0]:
# Saving the dataframe to memory for repetitive use
df.cache()
df.count()

2697

## STEP 2. Based on the above dataframe, prepare data from machine learning

#### 2.1. Text Cleaning Preprocessing

`pyspark.sql.functions.regexp_replace` is used to process the text

1. Remove URLs such as `http://stackoverflow.com`
2. Remove special characters
3. Substituting multiple spaces with single space
4. Lowercase all text
5. Trim the leading/trailing whitespaces

In [0]:
# Preprocessing the data
cleaned = df.withColumn('text', regexp_replace('text', r"http\S+", "")) \
                    .withColumn('text', regexp_replace('text', r"[^a-zA-z]", " ")) \
                    .withColumn('text', regexp_replace('text', r"\s+", " ")) \
                    .withColumn('text', lower('text')) \
                    .withColumn('text', trim('text')) 
# display(cleaned)


## STEP 3. Machine Learning Model Training

#### 3.1 Feature Transformer

##### 3.1.1 Tokenizer

In [0]:

from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
tokenized = tokenizer.transform(cleaned)

# display(tokenized)

##### 3.1.2 Stopword Removal

In [0]:
from pyspark.ml.feature import StopWordsRemover

stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
stopword = stopword_remover.transform(tokenized)

# display(stopword)

##### 3.1.3 CountVectorizer (TF - Term Frequency)

In [0]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(vocabSize=2**16, inputCol="filtered", outputCol='cv')
cv_model = cv.fit(stopword)
text_cv = cv_model.transform(stopword)

# display(text_cv)

##### 3.1.4 TF-IDF Vectorization

In [0]:
from pyspark.ml.feature import HashingTF, IDF

idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
idf_model = idf.fit(text_cv)
text_idf = idf_model.transform(text_cv)

# display(text_idf)

#### 3.2 Label Encoding

In [0]:
from pyspark.ml.feature import StringIndexer

label_encoder = StringIndexer(inputCol = "tags", outputCol = "label")
le_model = label_encoder.fit(text_idf)
final = le_model.transform(text_idf)

# display(final)

#### 3.4 Model Training

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=100)

lr_model = lr.fit(final)

predictions = lr_model.transform(final)

# display(predictions)

#### 3.5 Model Evalution

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
roc_auc = evaluator.evaluate(predictions)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(predictions.count())

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

Accuracy Score: 0.3482
ROC-AUC: 0.2841


#### 3.6 Create a Pipeline

In [0]:
# Importing all the libraries
from pyspark.sql.functions import split, translate, trim, explode, regexp_replace, col, lower
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Preparing the data
# Step 1: Creating the joined table
df = posts.join(postType, posts.PostTypeId == postType.id)
# Step 2: Selecting only Question posts
df = df.filter(col("Type") == "Question")
# Step 3: Formatting the raw data
df = (df.withColumn('Body', regexp_replace(df.Body, r'<.*?>', ''))
      .withColumn("Tags", split(trim(translate(col("Tags"), "<>", " ")), " "))
)
# Step 4: Selecting the columns
df = df.select(col("Body").alias("text"), col("Tags"))
# Step 5: Getting the tags
df = df.select("text", explode("Tags").alias("tags"))
# Step 6: Clean the text
cleaned = df.withColumn('text', regexp_replace('text', r"http\S+", "")) \
                    .withColumn('text', regexp_replace('text', r"[^a-zA-z]", " ")) \
                    .withColumn('text', regexp_replace('text', r"\s+", " ")) \
                    .withColumn('text', lower('text')) \
                    .withColumn('text', trim('text')) 

# Machine Learning
# Step 1: Train Test Split
train, test = cleaned.randomSplit([0.9, 0.1], seed=20200819)
# Step 2: Initializing the transfomers
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
cv = CountVectorizer(vocabSize=2**16, inputCol="filtered", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5)
label_encoder = StringIndexer(inputCol = "tags", outputCol = "label")
lr = LogisticRegression(maxIter=100)
# Step 3: Creating the pipeline
pipeline = Pipeline(stages=[tokenizer, stopword_remover, cv, idf, label_encoder, lr])
# Step 4: Fitting and transforming (predicting) using the pipeline
pipeline_model = pipeline.fit(train)
predictions = pipeline_model.transform(test)

## STEP 4. Save the Model file to Azure storage

In [0]:
# Saving model object to the /mnt/deBDProject directory. Yours name may be different.
pipeline_model.save('/mnt/deBDProject/model')

# Save the the String Indexer to decode the encoding. We need it in the future Sentiment Analysis.
le_model.save('/mnt/deBDProject/stringindexer')

In [0]:
# Review the directory
display(dbutils.fs.ls("/mnt/deBDProject/model"))

path name size modificationTime dbfs:/mnt/deBDProject/model/metadata/ metadata/ 0 1715588798000 dbfs:/mnt/deBDProject/model/stages/ stages/ 0 1715588798000

In the mean time, you can see the model as been mounted the Azure storage already.

<img src='https://s3.amazonaws.com/weclouddata/images/data_engineer/ml_tr113.jpg' width='35%'></div >

It is all done, let's use this model.

**Don't forget to turn off your Databricks computing cluster.**